<a href="https://colab.research.google.com/github/Marconiadsf/Desafio-4---I2A2/blob/main/Desafio4v11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DESAFIO 4 - Versão 11**

**Como usar:**
1) Faça upload desse arquivo e abra no seu google Colab
2) No icone da pasta ao lado suba os arquivos exceto o VR MENSAL 05.2025.xlsx
3) Crie uma APY KEY do Google e a variável de ambiente no icone de chave ai do lado. A variável deve se chamar: GOOGLE_API_KEY
4) Clique em Executar tudo na barra superior abaixo de ambiente de execução.
O arquivo VR MENSAL 05.2025.xlsx será criado na mesma pasta ao lado.

In [ ]:
!pip install google-adk -q
!pip install litellm -q
!pip install unidecode -q
print("Installation complete.")

Installation complete.


In [ ]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
import time
from unidecode import unidecode
from typing import List
from typing import Dict, Any
import re
import warnings



# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [ ]:
# Configura a API Key do Google Gemini
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# --- Define Model Constants for easier use ---

# More supported models can be referenced here: https://ai.google.dev/gemini-api/docs/models#model-variations
MODEL_GEMINI_2_5_FLASH = "gemini-2.5-flash"


print("\nEnvironment configured.")



Environment configured.


In [ ]:
# @title Define the dataframe tools
def criarColunaRegex(
    df_name: str,
    src_column_name: str,
    new_column_name: str,
    lista_de_strings: List[str]
) -> dict:
    """
    Extrai padrões de texto de uma coluna (especificada por nome) usando regex gerado a partir de uma lista de strings.

    Args:
        df_name (str): Nome do DataFrame no DataFrameManager.
        src_column_name (str): Nome da coluna de origem.
        new_column_name (str): Nome da nova coluna a ser criada.
        lista_de_strings (List[str]): Lista de padrões a serem extraídos (ex: siglas de estados).

    Returns:
        dict: {"status": "success", "df_name": <nome>, "error": None} ou {"status": "error", ...}
    """
    df = DataFrameManager.get(df_name)
    if df is None:
        return {
            "status": "error",
            "df_name": None,
            "error": f"DataFrame '{df_name}' não encontrado."
        }

    if src_column_name not in df.columns:
        return {
            "status": "error",
            "df_name": None,
            "error": f"Coluna '{src_column_name}' não existe em '{df_name}'."
        }

    try:
        # Monta o regex a partir da lista
        padrao = r"\b(" + "|".join(re.escape(s) for s in lista_de_strings) + r")\b"

        # Aplica a extração com conversão segura para string
        df[new_column_name] = df[src_column_name].astype(str).str.extract(padrao, expand=False)

        DataFrameManager.update({df_name: df})
        return {"status": "success", "df_name": df_name, "error": None}

    except Exception as e:
        return {
            "status": "error",
            "df_name": None,
            "error": str(e)
        }

def mapearValoresColunaNome(
    df_name: str,
    coluna_origem: str,
    coluna_destino: str,
    mapa_valores: Dict[Any, Any]
) -> dict:
    """
    Cria uma nova coluna em um DataFrame com base no mapeamento de valores da coluna existente.

    Args:
        df_name (str): Nome do DataFrame no DataFrameManager.
        coluna_origem (str): Nome da coluna a ser usada como base.
        coluna_destino (str): Nome da nova coluna a ser criada.
        mapa_valores (Dict[Any, Any]): Dicionário de mapeamento {valor_original: valor_novo}.

    Returns:
        dict: {"status": "success", "df_name": <nome>, "error": None} ou {"status": "error", ...}
    """
    df = DataFrameManager.get(df_name)
    if df is None:
        return {
            "status": "error",
            "df_name": None,
            "error": f"DataFrame '{df_name}' não encontrado."
        }

    if coluna_origem not in df.columns:
        return {
            "status": "error",
            "df_name": None,
            "error": f"Coluna '{coluna_origem}' não existe em '{df_name}'."
        }

    try:
        df[coluna_destino] = df[coluna_origem].map(mapa_valores)
        DataFrameManager.update({df_name: df})
        return {
            "status": "success",
            "df_name": df_name,
            "error": None
        }

    except Exception as e:
        return {
            "status": "error",
            "df_name": None,
            "error": str(e)
        }

def mapearValoresColunaIndex(
    df_name: str,
    coluna_origem_index: int,
    coluna_destino: str,
    mapa_valores: Dict[Any, Any]
) -> dict:
    """
    Cria uma nova coluna em um DataFrame com base no mapeamento de valores da coluna existente.

    Args:
        df_name (str): Nome do DataFrame no DataFrameManager.
        coluna_origem_index (int): Índice da coluna a ser usada como base.
        coluna_destino (str): Nome da nova coluna a ser criada.
        mapa_valores (Dict[Any, Any]): Dicionário de mapeamento {valor_original: valor_novo}.

    Returns:
        dict: {"status": "success", "df_name": <nome>, "error": None} ou {"status": "error", ...}
    """
    df = DataFrameManager.get(df_name)
    if df is None:
        return {"status": "error", "df_name": None, "error": f"DataFrame '{df_name}' não encontrado."}

    try:
        # Verifica se o índice da coluna é válido
        if coluna_origem_index < 0 or coluna_origem_index >= len(df.columns):
            return {"status": "error", "df_name": None, "error": f"Índice de coluna inválido: {coluna_origem_index}"}

        coluna_origem = df.columns[coluna_origem_index]

        # Aplica o mapeamento
        df[coluna_destino] = df[coluna_origem].map(mapa_valores)

        # Atualiza o DataFrame
        DataFrameManager.update({df_name: df})
        return {"status": "success", "df_name": df_name, "error": None}

    except Exception as e:
        return {"status": "error", "df_name": None, "error": str(e)}

def querydataframe(query: str, df_name: str, variables: dict) -> dict:
    """Executa uma query sobre um DataFrame usando pandas.query, com suporte a variáveis externas.

    Args:
        query (str): A string com a operação a ser realizada no dataframe.
        df_name (str): O nome do dataframe a ser utilizado.
        variables (dict, opcional): Dicionário com variáveis externas à query.
        verbose (bool, opcional): Se True, imprime logs de execução.

    Retorna:
        dict: Um dicionário com o resultado da query.
              Inclui uma chave 'status' ('success' ou 'error').
              Se 'success', inclui o nome do dataframe resultante na chave 'df_name' e None na chave 'error'.
              Se 'error', a chave 'df_name' é None e 'error' contém a mensagem de erro.
    """
    if variables is None:
      variables = {}
    df = DataFrameManager.get(df_name)
    if df is None:
        #print(f"--- Tool: querydataframe called: No dataframe {df_name} avaiable ---")
        return {"status": "error", "df_name": None, "error": "DataFrame does not exist."}

    #print(f"--- Tool: querydataframe called for dataframe: {df_name} ---")

    try:
        result = df.query(query, local_dict=variables)
        DataFrameManager.update({df_name: result})
        return {"status": "success", "df_name": df_name, "error": None}
    except Exception as e:
        return {"status": "error", "df_name": None, "error": str(e)}

def evaldataframe(expression: str, df_name: str, variables: dict) -> dict:
    """Executa uma expressão sobre o dataframe, através do método eval com suporte a variáveis externas.

    Args:
        code (str): A string com o código a ser executado.
        df_name (str): O nome do dataframe a ser utilizado.

    Retorna:
        dict: Um dicionário com o resultado da query.
              Inclui uma chave de 'status' ('success' or 'error').
              Se 'success', inclui o nome do dataframe resultante na chave 'df_name' e None na chave 'error'.
              Se 'error', a chave 'df_name' é None e 'error' contém a mensagem de erro.
    """
    if variables is None:
      variables = {}
    df = DataFrameManager.get(df_name)
    if df is None:
        #    print(f"--- Tool: evaldataframe called: No dataframe {df_name} available ---")
        return {"status": "error", "df_name": None, "error": "DataFrame does not exist."}

    #print(f"--- Tool: evaldataframe called for dataframe: {df_name} ---")

    try:
        result = df.eval(expr=expression, local_dict=variables)
        #new_df_name = f"{df_name}_newquery_{int(time.time())}"
        DataFrameManager.update({df_name: result})
        return {"status": "success", "df_name": df_name, "error": None}
    except Exception as e:
        return {"status": "error", "df_name": None, "error": str(e)}

def getdataframenames() -> list:
    return list(DataFrameManager.keys())

def getdataframeheader(df_name: str) -> dict:
    df = DataFrameManager.get(df_name)
    if df is None:
        return {}
    header = {col: str(dtype) for col, dtype in df.dtypes.items()}
    return  header

def getdataframestdjheader(df_name: str) -> str:
    df = DataFrameManager.get(df_name)
    if df is None:
        return {}
    header = df.head(5).to_json(orient="records", lines=True)
    return  header

def getdataframestdmheader(df_name: str) -> str:
    df = DataFrameManager.get(df_name)
    if df is None:
        return {}
    header = df.head(5).to_markdown()
    return  header

def adicionarDescricao(df_name: str, title: str, description: str) -> dict:
    df = DataFrameManager.get(df_name)
    if df is None:
        return {"status": "error", "df_name": None, "error": "DataFrame does not exist."}
    df.__setattr__('title', title)
    df.__setattr__('description', description)
    return {"status": "success", "df_name": df_name, "error": None}

def copiarColunaSimples(dest_df_name: str, src_df_name: str, dest_column_name: str, src_column_name: str) -> dict:
    """
    Copia os valores da coluna do DataFrame `src_df_name`
    e define como índice do DataFrame `dest_df_name`.

    Requisitos:
    - Ambos os DataFrames devem existir no DataFrameManager.
    - A coluna de matrícula deve existir no DataFrame de origem.

    Retorna:
        dict: {"status": "success", "df_name": <nome_do_df_destino>, "error": None}
              ou {"status": "error", "df_name": None, "error": <mensagem>}
    """
    df_dest = DataFrameManager.get(dest_df_name)
    df_src = DataFrameManager.get(src_df_name)

    if df_dest is None or df_src is None:
        return {"status": "error", "df_name": None, "error": "Um ou ambos os DataFrames não existem."}

    if src_column_name not in df_src.columns:
        return {"status": "error", "df_name": None, "error": f"Coluna '{src_column_name}' não encontrada em '{src_df_name}'."}

    try:
        # Define o índice do DataFrame destino com os valores da coluna de matrícula

        df_dest[dest_column_name] = df_src[src_column_name].values
        # Atualiza o DataFrame no DataFrameManager
        DataFrameManager[dest_df_name] = df_dest

        return {"status": "success", "df_name": dest_df_name, "error": None}

    except Exception as e:
        return {"status": "error", "df_name": None, "error": str(e)}


def copiarColunaAdicionando(dest_df_name, src_df_name, dest_column_name, src_column_name):
    """
    Copia os valores da coluna src_column_name do DataFrame src_df_name
    e os adiciona abaixo da coluna dest_column_name do DataFrame dest_df_name.

    Requisitos:
    - Ambos os DataFrames devem existir no DataFrameManager.
    - Ambas as colunas devem existir nos respectivos DataFrames.

    O resultado é um novo DataFrame com a coluna dest_column_name expandida,
    contendo os dados originais seguidos dos dados da coluna src_column_name.

    Retorna:
        dict: {"status": "success", "newdf": <nome_do_novo_df>, "error": None}
              ou {"status": "error", "newdf": None, "error": <mensagem>}
    """


    # Recupera os DataFrames
    dest_df = DataFrameManager.get(dest_df_name)
    src_df = DataFrameManager.get(src_df_name)

    if dest_df is None or src_df is None:
        return {"status": "error", "df_name": None, "error": "Um ou ambos os DataFrames não existem."}

    # Verifica se as colunas existem
    if dest_column_name not in dest_df.columns:
        return {"status": "error", "df_name": None, "error": f"Coluna '{dest_column_name}' não encontrada em '{dest_df_name}'."}
    if src_column_name not in src_df.columns:
        return {"status": "error", "df_name": None, "error": f"Coluna '{src_column_name}' não encontrada em '{src_df_name}'."}

    try:
        # Extrai as colunas como Series
        coluna_dest = dest_df[dest_column_name]
        coluna_src = src_df[src_column_name]

        # Concatena verticalmente
        coluna_combinada = pd.concat([coluna_dest, coluna_src], ignore_index=True)

        # Atualiza a coluna no DataFrame de destino
        dest_df_expanded = dest_df.copy()
        dest_df_expanded = dest_df_expanded.reindex(range(len(coluna_combinada)))
        dest_df_expanded[dest_column_name] = coluna_combinada

        # Atualiza o DataFrame no DataFrameManager
        DataFrameManager[dest_df_name] = dest_df_expanded

        return {"status": "success", "newdf": dest_df_name, "error": None}

    except Exception as e:
        return {"status": "error", "newdf": None, "error": str(e)}


def fundirColunaBaseIndexador(df_dest_name: str, df_src_name: str, index_column_name: str, column_name_list: List[str]) -> dict:
    dfdest = DataFrameManager.get(df_dest_name)
    dfsource = DataFrameManager.get(df_src_name)

    if dfdest is None:
        return {"status": "error", "df_name": None, "error": "DataFrame does not exist."}
    if dfsource is None:
        return {"status": "error", "df_name": None, "error": "DataFrame does not exist."}

    try:
        # Garante que todas as colunas existem no dfsource
        colunas_para_copiar = [index_column_name] + column_name_list
        for col in colunas_para_copiar:
            if col not in dfsource.columns:
                return {"status": "error", "df_name": None, "error": f"Coluna '{col}' não encontrada em '{df_src_name}'."}

        # Executa o merge
        dfdest = pd.merge(
            dfdest,
            dfsource[colunas_para_copiar],
            on=index_column_name,
            how='left'
        )

        DataFrameManager.update({df_dest_name: dfdest})
        return {"status": "success", "df_name": df_dest_name, "error": None}

    except Exception as e:
        return {"status": "error", "df_name": None, "error": str(e)}

def padronizarNomes():
  # Strip all whitespace from all column names in the DataFrames
  for key, df in DataFrameManager.items():
     df.columns = [unidecode(col.strip()).upper() for col in df.columns]

def removerLixoCabecalhos(df_name: str) -> dict:
  df = DataFrameManager.get(df_name)
  if df is None:
    return {"status": "error", "df_name": None, "error": "DataFrame does not exist."}
  # Remove linhas com lixo no começo de um dataframe transformando a segunda linha em cabeçalho.
  df.columns = df.iloc[0]  # Usa a segunda linha como cabeçalho
  df = df.iloc[1:].reset_index(drop=True)  # Remove as duas primeiras linhas e reseta o índice
  DataFrameManager.update({df_name: df})
  return {"status": "success", "df_name": df_name, "error": None}

def renameColumn(df_name: str, name_dict: dict) -> dict:
  df = DataFrameManager.get(df_name)
  if df is None:
    return {"status": "error", "df_name": None, "error": "DataFrame does not exist."}
  df.rename(columns=name_dict, inplace=True)
  return {"status": "success", "df_name": df_name, "error": None}

In [ ]:
operadorDataFrameInstrucoes = """
Você é um analista de dados experiente em uma equipe colaborativa.

Objetivo:
- Responder solicitações sobre DataFrames usando exclusivamente as ferramentas: querydataframe, evaldataframe, getdataframenames e getdataframeheader.
- Retornar apenas um dicionário no formato padrão da equipe. Não inclua explicações textuais.

Formato padrão de resposta e outras regras:
- Sucesso: {"status": "success", "newdf": "<nome_do_dataframe_resultante>", "error": None}
- Erro: {"status": "error", "newdf": None, "error": "<mensagem_de_erro>", "attempts": [...]}  # "attempts" é opcional e lista as tentativas de correção.
- Todos os nomes de colunas criadas deverão ser maiúsculas.


Ferramentas:
- querydataframe(query: str, df_name: str, variables: dict)
  Use para filtros/seleção de linhas via expressões booleanas (pandas.query).
- evaldataframe(expression: str, df_name: str, variables: dict)
  Use para criar/modificar colunas via expressões matemáticas/lógicas (pandas.eval).
- criarColunaRegex(df_name: str,src_column_name: src,new_column_name: str,lista_de_strings: List[str])
  Uma função capaz de criar uma nova coluna, com base numa busca regex da src_column_name, pela string ou strings em lista_de_strings
- mapearValoresColunaNome(df_name: str,coluna_origem: str,coluna_destino: str,mapa_valores: Dict[Any, Any])
  Essa função recebe um coluna do dataframe selecionado e cria uma nova coluna com base no mapeamento de valores.
- getdataframenames()
  Retorna uma lista simples com os nomes dos DataFrames disponíveis.
- getdataframeheader(df_name: str)
  Retorna um dict com nomes de colunas e tipos do DataFrame informado.

Variáveis externas:
- Dentro de query/expression, variáveis externas devem ser referenciadas com @ (ex.: Fee > @min_fee).
- Sempre passe o dicionário "variables" correspondente (ex.: {"min_fee": 23000}).

Resolução do nome do DataFrame (obrigatório antes de operar):
1) Consulte a ferramenta getdataframenames()
2) Verifique se o nome que o usuário citou de dataframe ou tabela está presente na lista e o use.
3) Se o dataframe for inexiste repasse o erro sem inventar resultados.
4) Jamais tente acessar DataFrames diretamente; sempre use os nomes e as ferramentas.

Seleção de ferramenta:
- Use querydataframe para filtrar/selecionar linhas (condições booleanas).
- Use evaldataframe para criar/modificar colunas ou realizar cálculos.
- Combine variáveis externas com @ quando necessário.

Casos especiais:
- Para criação de colunas com busca regex, ou mencionado algo como extracao de uma valor de uma coluna use a ferramenta criarColunaRegex.
  Exemplo: criarColunaRegex("DataframeDestino","ColunaFonte","NovaColuna",["Foo", "Bar", "Fee", "Beer"])
  Ela extraí o valor da coluna fonte selecionada e cria uma nova coluna
- Se for solicitado para criar uma coluna com base nos valores de outra coluna use mapear_valores_em_coluna.
  Exemplo: mapearValoresColunaNome("DataframeDestino","ColunaFonte","NovaColuna",{"Foo": 1, "Bar": 2, "Fee": 3, "Beer": 4})
  Você deve criar o dicionário com base na requisição do usuário.

Resoluçao de consultas, erros e regras para reiterações de consultas (até 3 tentativas):

1) Consulte os nomes das colunas do dataframe com getdatraframeheader()
2) Verifique se usuário faz uma menção especifica a nome de coluna ela está na lista. Se não tente deduzir a coluna mais provável usando a seguinte lógica:
    - Busque o nome mais próximo por similaridade óbvia (ex.: Vemdas -> Vendas), incluindo:
     * Correção de acentos e pequenas trocas de letras, minúsculas e maiúsculas.
     * Se não houver candidato plausível, retorne um erro.
     LEMBRETE: O erro pode ser tant ode digitação do usuário quanto da planilha original.
3) Faça a consulta na coluna inferida usando a ferramenta apropriada
4) Em caso de erro:
  - Se o erro indicar problema na query/expression (ex.:sintaxe inválida):
     --Ajuste a query/expression (use notnull()/isnull(), operadores válidos, etc.) - lembre-se que getdatraframeheader() retorna também os tipos de dados.
  - Tente refazer a chamada com as correções até 3 vezes no total.
  - Registre cada tentativa falha no campo "attempts": [{"tool": "...", "error": "..."}, ...].
  - Após 3 tentativas sem sucesso, retorne o último erro.

Boas práticas de query/expression:
- Para nulos, use isnull() e notnull().
- Prefira referências claras de coluna e parênteses em expressões compostas.
- Toda variável externa deve usar @ e estar em "variables".

Exemplos:
- Filtro por taxa mínima:
  querydataframe("Fee > @min_fee", "Cursos", variables={"min_fee": 23000})
- Nova coluna com desconto adicional:
  evaldataframe("final_fee = (Fee - Discount) * (1 - @extra_discount)", "Cursos", variables={"extra_discount": 0.05})

Resposta:
- Sempre retorne exclusivamente o dicionário no formato padrão.
Observação importante:
- Mesmo que a solicitação pareça pedir um único valor (ex.: preço de um curso, nome de um cliente), considere isso como uma solicitação de filtragem do DataFrame.
Use a ferramenta apropriada para retornar um novo DataFrame contendo essas informações. Não trate esses casos como erro.
"""

In [ ]:
descritorDataFrameInstrucoes = """
Você é um analista de dados experiente em uma equipe colaborativa.

Objetivo:
Utilize suas ferramentas para gerar descrições sobre um dataframe.
Descubra quantos dataframes devem ser analisados usando a ferramenta getdataframenames: essa ferramenta retorna uma lista com os nomes dos dataframes disponíveis.
Analise o cabeçalho de cada dataframe e o tipo de dados com a ferramenta getdataframeheader: Essa ferramenta retorna um dicionário com os nomes das colunas e seus tipos.
De acordo com os nomes de colunas encontradas e tipos de dados tende determinar do que se trata o dataframe com uma frase simples.
Utilize o prompt de entrada para mais contexto.

Descrição das ferramentas:

- getdataframenames()
  Retorna uma lista simples com os nomes dos DataFrames disponíveis.
- getdataframeheader(df_name: str)
  Retorna um dict com nomes de colunas e tipos do DataFrame informado.
- adicionarDescricao(df_name: str, title: str, description: str)
  Adiciona um titulo e uma descrição ao dataframe. Retorna:
  - {"status": "error", "newdf": None, "error": "DataFrame does not exist."} em caso de erro ou
  - {"status": "success", "newdf": df_name, "error": None} em caso de sucesso.

Exemplos:
  Prompt de entrada de alguem da equipe: "Analise esses folhas de dados de vales refeiçao."
  Suas ações:
  1. Chamar getdataframesnames(). Retorno da função (exemplo): ['Ativos','Sindicatos','Demitidos','Admitidos Abril' ]
  2. Chamar getdataframeheader('Ativos'). Retorno da função (exemplo): {'Matricula': 'int64','Cargo': 'object','Sindicato': 'object', 'Situacao': 'object', 'Unammed1': 'object'}
  3. Com base nessa informação você pode inferir corretamente que estamos falando de um dataframe que, dados o prompt sobre vales refeição, provavelmente descreve uma lista de
  colaboradores ativos de uma empresa contendo informações como matrícula, cargo, sindicato, situação e uma coluna não nomeada.
  4. Crie um titulo e uma descriçao para o dataframe derivados de sua análise. Mantenha o titulo até 32 caracteres e a descricao até 200 caracteres.
  5. Terminada a análise voce deve chamar a funcao _____(df_name,title,description) para reportar sua análise. Para o exemplo acima ficaria:
    _____('Ativos','Lista de Funcionarios Ativos','Lista de colaboradores ativos. Contém as informações: matrícula, cargo, sindicato, situação e uma coluna não nomeada.')
  6. Você deve fazer isso para os demais dataframes retornados pela função getdataframesnames().
  7. Não tente acessar dataframes por sua conta, utilize apenas essas ferramentas.
  8. Seja fiel ao conteúdo e não invente. Se uma planilha não contiver dados que ajudem a identifica-la frente ao contexto, seja sincero como por exemplo:
    "Planilha com dados não (estruturados/nomeados/identificados) dentro do contexto de vale refeição. Possui colunas: numérica(int64), string?(objeto)"
  9. Observe que o a instrucão 8 serve para quando a função getdataframeheader retornar valores estranhos de uma planilha nao formatada.

"""

In [ ]:
formatadorInstrucoes = """
Você é um analista de dados experiente e disciplinado, responsável por padronizar colunas de DataFrames com base em instruções explícitas do usuário.

Seu papel é executar, não interpretar.

Objetivo:
Padronizar os nomes de colunas dos DataFrames disponíveis, corrigindo erros de digitação, inconsistências e variações semânticas.

Ferramentas disponíveis:
padronizarnomes() Limpa espaços, acentos, caracteres especiais e padroniza para maiúsculas em todos os DataFrames.
removerLixoCabecalhos(df_name: str) - Remove linhas com lixo no começo de um dataframe transformando a segunda linha em cabeçalho. Cuidado, só chame essa função quando um cabeçalho paracer uma tabela pouco ou mal estruturada
getdataframenames() Retorna os nomes dos DataFrames disponíveis.

getdataframeheader(df_name) Retorna os nomes e tipos das colunas.

getdataframestdmheader(df_name) Retorna as primeiras 5 linhas em formato markdown.

renameColumn(df_name, name_dict) Renomeia colunas com base em um dicionário de equivalência.

Regras obrigatórias:
Execute padronizarnomes() antes de qualquer inspeção.

Use getdataframenames() para listar os DataFrames disponíveis.

Para cada DataFrame:

Use getdataframeheader() e getdataframestdmheader() para entender a estrutura.

Se o cabeçalho estiver ilegível ou não confiável, por exemplo, com colunas que indiquem um campo numérico mas o primeiro item observado é texto,
utilize a removerLixoCabecalhos(nome do dataframe), pois pode ser que a primeira linha contenha lixo e a segunda seja o cabeçalho real.

Renomeie colunas com base nas instruções do usuário, usando renameColumn():
Exemplos:
Colunas como CADASTRO, CADASTRO FUNCIONAL, CADASTRO DO FUNCIONARIO, CADASTRO DE PESSOAL, REGISTRO, MATRICULA FUNCIONAL → devem ser renomeadas para "MATRICULA".

Colunas como TITULO DO CARGO, CARGO, FUNÇÃO, DESIGNAÇÃO, NOME DO CARGO → devem ser renomeadas para "CARGO".

Não use acentos ou caracteres especiais nos nomes finais.

Corrija proativamente pequenos erros de digitação nos cabeçalhos como: VEMDAS -> VENDAS, SINDICADO -> SINDICATO, CODEGO -> CODIGO. MAS: NAO USE ACENTOS!!!!

Importante:
Não invente nomes de colunas.

Não tente acessar os dados diretamente — use apenas as ferramentas disponíveis.

Não peça confirmação do usuário para regras já definidas.

Execute as renomeações de forma proativa e completa.

Se algum DataFrame não puder ser processado, ignore e siga para o próximo. Ao final, informe os DataFrames que foram modificados e quais colunas foram renomeadas.

"""

In [ ]:
consolidadorInstrucoes = """
Você é um analista de dados experiente em uma equipe colaborativa.

Objetivo:
Consolide as informações de vários dataframes em um único dataframe.

Descrição das ferramentas:

- copiarColunaSimples(dest_df_name: str, src_df_name: str, dest_column_name: str, src_column_name: str) -> dict:
  Função para copiar os valores de um dataframe em src_column_name para o dataframe dest_df_name com o nome dest_column_name.
- fundirColunaBaseIndexador(df_dest_name: str, df_src_name: str, index_column_name: str, columns_names_list: List[str]):
  Função para copiar os valores das colunas columns_names do DataFrame src_df_name no dest_column_name
  com base num indexador dado por index_column_name.
- getdataframenames()
  Retorna uma lista simples com os nomes dos DataFrames disponíveis.
- getdataframeheader(df_name: str)
  Retorna um dict com nomes de colunas e tipos do DataFrame informado. Ou vazio em caso de não existir.


Entendedo a demanda do usuário: O usuário solicita "Busque as informações de cargo e situação para os colaboradores":
1. Você comecaria analisando quais tabelas tem informacao de cargo e possui a informacao de matricula rodando:
  -> getdataframenames() -> para saber quantos dataframes existem para ser analisados (ignorando, obviamente o DataFrameFinal)
  -> Para cada dataframe chame getdataframeheader(df_name: str) e verifique apartir dos nomes das colunas se é plausivel executar o passo 7 abaixo para ele.
2. Chamar a funcao fundirColunaBaseIndexador(df_dest_name: str, df_src_name: str,index_column_name: str, target_column_name:dict) para os dataframes selecionados, um por vez.
  Para o exemplo citado acima, seria:
  Pegando as informacoes de 'CARGO' e 'DESC. SITUACAO' no dataframe 'ATIVOS':
        fundirColunaBaseIndexador('DataFrameFinal','ATIVOS','MATRICULA',['CARGO', 'DESC. SITUACAO'])
  Faça similar para outros dataframes.
  Se alguma coluna já tiver o mesmo nome no DataFrameFinal, simplesmente adicione sulfixos de uma letra _X, _Y, _Z etc..., verfique por que o pandas faz isso entao alguns sulfixos podem ja existir.
ATENÇÃO:

  - Não tente acessar dataframes por sua conta, utilize apenas essas ferramentas.

"""

In [ ]:
# @title Define o agente operador de dataframe
# Use one of the model constants defined earlier
AGENT_MODEL = MODEL_GEMINI_2_5_FLASH # Starting with Gemini

agenteOperadorDataFrame = Agent(
    name="OperadorDataFrame",
    model=AGENT_MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Realiza operações em dataframes usando as ferramentas df.query ou df.eval, de acordo com a complexidade da tarefa.",
    instruction=operadorDataFrameInstrucoes,
    tools=[querydataframe,evaldataframe, getdataframenames,getdataframeheader,criarColunaRegex,mapearValoresColunaNome], # Pass the function directly
)


In [ ]:
# @title Define o agente descritor de dataframe
# Use one of the model constants defined earlier
AGENT_MODEL = MODEL_GEMINI_2_5_FLASH # Starting with Gemini

agenteDescritorDataFrame = Agent(
    name="DescritorDataFrameV1",
    model=AGENT_MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Descreve o conteúdo de um pandas DataFrame em linguagem natural, através de reasoning e uso de ferramentas.",
    instruction=descritorDataFrameInstrucoes,
    tools=[getdataframenames,getdataframeheader,adicionarDescricao], # Pass the function directly
)

#print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")


In [ ]:
# @title Define o agente formatador
# Use one of the model constants defined earlier
AGENT_MODEL = MODEL_GEMINI_2_5_FLASH # Starting with Gemini

agenteFormatadorDataFrame = Agent(
    name="FormatadorDataFrameV1",
    model=AGENT_MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Agente para fazer algumas correçoes e padronizações simples em dataframes problemáticos",
    instruction=formatadorInstrucoes,
    tools=[padronizarNomes,getdataframenames,getdataframeheader,getdataframestdjheader,getdataframestdmheader,renameColumn,removerLixoCabecalhos], # Pass the function directly
)

#print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")


In [ ]:
# @title Define o agente consolidador
# Use one of the model constants defined earlier
AGENT_MODEL = MODEL_GEMINI_2_5_FLASH # Starting with Gemini

agenteConsolidadorDataFrame = Agent(
    name="ConsolidadorDataFrameV1",
    model=AGENT_MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Agente para iniciar a consolidacao do dataframe final",
    instruction=consolidadorInstrucoes,
    tools=[copiarColunaSimples,fundirColunaBaseIndexador,getdataframenames,getdataframeheader], # Pass the function directly
)

#print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")


In [ ]:
# @title Setup Session Service and Runner

# --- Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
sessionOperador = InMemorySessionService()
sessionDescritor = InMemorySessionService()
sessionFormatador = InMemorySessionService()
sessionConsolidador = InMemorySessionService()
# --- Inicialização de múltiplos agentes e Runners ---

acesso = {"Operador": 0, "Descritor": 1,"Formatador": 2,"Consolidador": 3}
# Listas de identificação
app_id_list = ["OperadorDataFrame", "DescritorDataFrame","FormatadorDataFrame","ConsolidadorDataFrame"]
user_id_list = ["operador", "descritor","formatador","consolidador"]
session_id_list = ["sessao_operador", "sessao_descritor","sessao_formatador","sessao_consolidador"]
agent_list = [agenteOperadorDataFrame, agenteDescritorDataFrame,agenteFormatadorDataFrame,agenteConsolidadorDataFrame]
session_service_list = [sessionOperador, sessionDescritor,sessionFormatador,sessionConsolidador]

# Armazenamento de sessões e Runners
session_list = []
runner_list = []

# Criação das sessões e dos Runners correspondentes
for app_name, user_id, session_id, agent,session_service in zip(app_id_list, user_id_list, session_id_list, agent_list,session_service_list):
    session = await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id
    )
    session_list.append(session)

    runner = Runner(
        agent=agent,
        app_name=app_name,
        session_service=session_service
    )
    runner_list.append(runner)

In [ ]:
# @title Define Agent Interaction Function

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      #print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [ ]:
# @title Cria o DataFrameManager e carrega arquivos na memória (subir aqrquivos ai no lado se estiver no colab - Não Subir o Modelo de tabela final.. "VR ...")

DataFrameManager = {

}

def list_files_in_directory(directory_path):
    """
    Lists all files in a given directory.

    Args:
        directory_path (str): The path to the directory.

    Returns:
        list: A list of filenames in the directory.
              Returns an empty list if the directory does not exist or an error occurs.
    """
    if not os.path.isdir(directory_path):
        print(f"Error: The provided path '{directory_path}' is not a valid directory.")
        return []

    try:
        # os.listdir() returns a list containing the names of the entries in the directory
        # The list is in arbitrary order.
        all_entries = os.listdir(directory_path)

        # We need to filter for files and exclude directories
        files = [entry for entry in all_entries if os.path.isfile(os.path.join(directory_path, entry))]

        return files
    except OSError as e:
        print(f"An error occurred: {e}")
        return []

def is_excel_file(filename):
    """
    Checks if a file is an Excel spreadsheet based on its extension.

    Args:
        filename (str): The name of the file to check.

    Returns:
        bool: True if the file has a common Excel extension, False otherwise.
    """
    # Get the file extension
    file_extension = os.path.splitext(filename)[1].lower()

    # Check if the extension is in our list of common Excel extensions
    excel_extensions = ['.xlsx', '.xls', '.xlsm']

    return file_extension in excel_extensions

current_dir = os.getcwd()
target_dir = os.path.join(current_dir, "")
files_list = list_files_in_directory(target_dir)
for file in  files_list:
    if is_excel_file(file):
        df = pd.read_excel(os.path.join(target_dir, file))
        if df is not None:
                           DataFrameManager.update({(os.path.splitext(file)[0]): df})
        else:
                           print(f"Error while trying to create a new dataframe from file {os.path.join(target_dir, file)}")


In [ ]:
# @title Iniciando o Tratamento dos Arquivos com Agentes:

# Iniciando o Agente formatador para trabalhar os nomes das colunas:
# To do: Se organizar a execucao deles em um loop poderia rodar a funcao de padronizacao primaria fora do agente reduzindo a carga de processamento: Ela é universal e não precisa do agente...
#        Já as funções posteriores precisam de raciocínio para serem ativadas corretamente.

async def run_conversation():
    # Chama o formatador reforçando algumas informações de conhecimento prévio
    await call_agent_async("Padronize os Dataframes disponíveis. Regras para desambiguação de nomes: Cadastro: renomeie para 'MATRICULA'. TITULO DO CARGO e variantes disso:  renomeie para 'CARGO'. IMPORTANTE: Lembre-se de corrigir erros de português",
                                       runner=runner_list[acesso['Formatador']],
                                       user_id=user_id_list[acesso['Formatador']],
                                       session_id=session_id_list[acesso['Formatador']])
    # Algumas vezes ele tem preguiça e deixa passar erros de português... por garantia vamos pedir de novo:
    await call_agent_async("Cheque mais uma vez os cabeçalhos buscando erros de poertuguês",
                                       runner=runner_list[acesso['Formatador']],
                                       user_id=user_id_list[acesso['Formatador']],
                                       session_id=session_id_list[acesso['Formatador']])
    # Chama o operador para fazer algumas operações com colunas (removido as ferramentas query e eval que não estavam ajudando)
    await call_agent_async("Para 'Base dias uteis': Crie uma coluna 'ESTADO_SIGLA'com base no nome do sindicato que pode ser encontrado na coluna 'SINDICATO'. Realize a extração da sigla do nome do sindicato. Use apenas o dataframe 'Base dias uteis' NÃO opere sobre o DataFrameFinal.",
                                       runner=runner_list[0],
                                       user_id=user_id_list[0],
                                       session_id=session_id_list[0])
    # Como estava tendo problemas de chaves, e também para evitar alucinações vamos quebrar os pedidos em etapas (talvez fosse interessante resetar o contexto aqui? -> tokens????) (Cada agente tem sua in memory session isolada....)
    await call_agent_async("Para 'Base dias uteis': Crie a coluna ESTADO com base na coluna 'ESTADO_SIGLA' para o nome de estado. Exemplo: PR -> Paraná. Use apenas o dataframe 'Base dias uteis' NÃO opere sobre o DataFrameFinal.",
                                       runner=runner_list[0],
                                       user_id=user_id_list[0],
                                       session_id=session_id_list[0])
    # Agora chamamos o consolidador com um set de ferramentas diferente para juntas informações: (Nota: Removi as funcoes para iniciar a criação do dataframe final ele estava se confundindo e copiando informacao no DataFrameFinal)
    await call_agent_async("Funda os dados de 'Base sindicato x valor' no dataframe 'Base dias uteis', trazendo a coluna 'VALOR' e tendo a coluna 'ESTADO' como indexador/coluna em comum. Use apenas o dataframe 'Base dias uteis' NÃO opere sobre o DataFrameFinal.",
                                       runner=runner_list[3],
                                       user_id=user_id_list[3],
                                       session_id=session_id_list[3])


# Execute the conversation using await in an async context (like Colab/Jupyter)

await run_conversation()


>>> User Query: Padronize os Dataframes disponíveis. Regras para desambiguação de nomes: Cadastro: renomeie para 'MATRICULA'. TITULO DO CARGO e variantes disso:  renomeie para 'CARGO'. IMPORTANTE: Lembre-se de corrigir erros de português


<<< Agent Response: DataFrames modificados e colunas renomeadas:

*   **EXTERIOR**: `CADASTRO` -> `MATRICULA`
*   **APRENDIZ**: `TITULO DO CARGO` -> `CARGO`
*   **Base dias uteis**:
    *   `removerLixoCabecalhos()` foi aplicado.
    *   `DIAS UTEIS ` -> `DIAS UTEIS`
*   **ATIVOS**: `TITULO DO CARGO` -> `CARGO`
*   **ESTÁGIO**: `TITULO DO CARGO` -> `CARGO`

>>> User Query: Cheque mais uma vez os cabeçalhos buscando erros de poertuguês


<<< Agent Response: A coluna `SINDICADO` no DataFrame `Base dias uteis` foi renomeada para `SINDICATO`.

>>> User Query: Para 'Base dias uteis': Crie uma coluna 'ESTADO_SIGLA'com base no nome do sindicato que pode ser encontrado na coluna 'SINDICATO'. Realize a extração da sigla do nome do sindicato. Use apenas o dataframe 'Base dias uteis' NÃO opere sobre o DataFrameFinal.


<<< Agent Response: {"status": "success", "newdf": "Base dias uteis", "error": None}

>>> User Query: Para 'Base dias uteis': Crie a coluna ESTADO com base na coluna 'ESTADO_SIGLA' para o nome de estado. Exemplo: PR -> Paraná. Use apenas o dataframe 'Base dias uteis' NÃO opere sobre o DataFrameFinal.


<<< Agent Response: {"status": "success", "newdf": "Base dias uteis", "error": None}

>>> User Query: Funda os dados de 'Base sindicato x valor' no dataframe 'Base dias uteis', trazendo a coluna 'VALOR' e tendo a coluna 'ESTADO' como indexador/coluna em comum. Use apenas o dataframe 'Base dias uteis' NÃO opere sobre o DataFrameFinal.


<<< Agent Response: As informações da coluna 'VALOR' do dataframe 'Base sindicato x valor' foram fundidas com sucesso no dataframe 'Base dias uteis', utilizando 'ESTADO' como indexador.


In [ ]:
# @title Mais funções auxiliares
################################################################################
# Função para automatizar um merge de info de dois dataframes com copia de todas as colunas

################################################################################

def simplemerge(
    df_dest: pd.DataFrame,
    df_source: pd.DataFrame,
    chave: str,
    how: str = "left"
) -> pd.DataFrame:
    """
    Faz merge entre df_dest e df_source, renomeando colunas duplicadas do df_source com sufixos numerados.

    Args:
        df_dest (pd.DataFrame): DataFrame principal.
        df_source (pd.DataFrame): DataFrame complementar.
        chave (str): Coluna usada como chave de junção.
        how (str): Tipo de merge (default: 'left').

    Returns:
        pd.DataFrame: Resultado do merge com colunas renomeadas dinamicamente.
    """
    if chave not in df_dest.columns or chave not in df_source.columns:
        print(f"[AVISO] Chave '{chave}' não encontrada em ambos os DataFrames.")
        return df_dest

    # Renomeia colunas duplicadas do df_source
    colunas_renomeadas = {}
    for col in df_source.columns:
        if col == chave:
            continue
        novo_nome = col
        contador = 1
        while novo_nome in df_dest.columns or novo_nome in colunas_renomeadas.values():
            novo_nome = f"{col}_{contador}"
            contador += 1
        colunas_renomeadas[col] = novo_nome

    df_source_renomeado = df_source.rename(columns=colunas_renomeadas)

    return pd.merge(df_dest, df_source_renomeado, on=chave, how=how)

################################################################################
# Funções para fazer merge de colunas repedidas...
# A primeira checa o tipo, a segunda não.
# Quando for usar com um agente melhor usar a primeira
# Criar outro agente para usar a segunda só nas colunas tipo anotação
# Mas para isso o agente descritor tem que estar 100% operacional e
# conseguir descrever colunas também.
###############################################################################

def consolidar_colunas(
    nome_data_frame: str,
    nome_coluna_destino: str,
    colunas_a_consolidar: List[str],
    operador: str
) -> dict:
    """
    Consolida colunas de um DataFrame usando operador definido, com tratamento para texto e números.
    Após a consolidação, remove as colunas originais utilizadas e renomeia a coluna gerada.

    Args:
        nome_data_frame (str): Nome do DataFrame no DataFrameManager.
        nome_coluna_destino (str): Nome da coluna final consolidada.
        colunas_a_consolidar (List[str]): Lista de colunas a consolidar.
        operador (str): Operador de consolidação ('|', '+', '-', '*', '/').

    Returns:
        dict: {"status": "success", "df_name": <nome>, "error": None} ou {"status": "error", ...}
    """
    df = DataFrameManager.get(nome_data_frame)
    if df is None:
        return {"status": "error", "df_name": None, "error": f"DataFrame '{nome_data_frame}' não encontrado."}

    try:
        # Verifica se todas as colunas existem
        colunas_invalidas = [col for col in colunas_a_consolidar if col not in df.columns]
        if colunas_invalidas:
            return {"status": "error", "df_name": None, "error": f"Colunas não encontradas: {colunas_invalidas}"}

        # Cria nome temporário seguro
        nome_temporario = f"__temp_{nome_coluna_destino}__"
        while nome_temporario in df.columns:
            nome_temporario += "_"

        # Detecta tipo dominante (texto ou numérico)
        tipos = df[colunas_a_consolidar].dtypes
        if all(tipos.apply(lambda t: pd.api.types.is_string_dtype(t))):
            # Consolidação textual
            def concat_texto(row):
                valores = [str(row[col]).strip() for col in colunas_a_consolidar
                           if pd.notnull(row[col]) and str(row[col]).strip() != ""]
                return operador.join(valores) if valores else np.nan

            df[nome_temporario] = df.apply(concat_texto, axis=1)

        elif all(tipos.apply(lambda t: pd.api.types.is_numeric_dtype(t))):
            # Consolidação numérica
            if operador == "+":
                df[nome_temporario] = df[colunas_a_consolidar].sum(axis=1, skipna=True)
            elif operador == "*":
                df[nome_temporario] = df[colunas_a_consolidar].prod(axis=1, skipna=True)
            elif operador == "-":
                df[nome_temporario] = df[colunas_a_consolidar].iloc[:, 0]
                for col in colunas_a_consolidar[1:]:
                    df[nome_temporario] -= df[col].fillna(0)
            elif operador == "/":
                df[nome_temporario] = df[colunas_a_consolidar].iloc[:, 0]
                for col in colunas_a_consolidar[1:]:
                    df[nome_temporario] = df[nome_temporario] / df[col].replace(0, np.nan)
            else:
                return {"status": "error", "df_name": None, "error": f"Operador numérico inválido: '{operador}'"}
        else:
            return {"status": "error", "df_name": None, "error": "Tipos mistos: não é possível consolidar texto com números."}

        # Remove colunas originais
        df.drop(columns=colunas_a_consolidar, inplace=True)

        # Renomeia coluna temporária para destino
        df.rename(columns={nome_temporario: nome_coluna_destino}, inplace=True)

        DataFrameManager.update({nome_data_frame: df})
        return {"status": "success", "df_name": nome_data_frame, "error": None}

    except Exception as e:
        return {"status": "error", "df_name": None, "error": str(e)}

def consolidar_colunas_como_texto(
    nome_data_frame: str,
    nome_coluna_destino: str,
    colunas_a_consolidar: List[str],
    operador: str
) -> dict:
    """
    Consolida colunas de um DataFrame ignorando tipos, convertendo tudo para string e concatenando com operador.
    Após a consolidação, remove as colunas originais utilizadas e renomeia a coluna gerada.

    Args:
        nome_data_frame (str): Nome do DataFrame no DataFrameManager.
        nome_coluna_destino (str): Nome da coluna final consolidada.
        colunas_a_consolidar (List[str]): Lista de colunas a consolidar.
        operador (str): Operador de concatenação (ex: '|', '-', etc.).

    Returns:
        dict: {"status": "success", "df_name": <nome>, "error": None} ou {"status": "error", ...}
    """
    df = DataFrameManager.get(nome_data_frame)
    if df is None:
        return {"status": "error", "df_name": None, "error": f"DataFrame '{nome_data_frame}' não encontrado."}

    try:
        # Verifica se todas as colunas existem
        colunas_invalidas = [col for col in colunas_a_consolidar if col not in df.columns]
        if colunas_invalidas:
            return {"status": "error", "df_name": None, "error": f"Colunas não encontradas: {colunas_invalidas}"}

        # Cria nome temporário seguro
        nome_temporario = f"__temp_{nome_coluna_destino}__"
        while nome_temporario in df.columns:
            nome_temporario += "_"

        # Consolidação forçada como texto
        def concat_forcado(row):
            valores = [str(row[col]).strip() for col in colunas_a_consolidar if pd.notnull(row[col])]
            return operador.join(valores) if valores else ""

        df[nome_temporario] = df.apply(concat_forcado, axis=1)

        # Remove colunas originais
        df.drop(columns=colunas_a_consolidar, inplace=True)

        # Renomeia coluna temporária para destino
        df.rename(columns={nome_temporario: nome_coluna_destino}, inplace=True)

        DataFrameManager.update({nome_data_frame: df})
        return {"status": "success", "df_name": nome_data_frame, "error": None}

    except Exception as e:
        return {"status": "error", "df_name": None, "error": str(e)}


################################################################################
# Função para juntar todos os dados em todos os dataframes que possuam o mesmo
# indexador.
# Aqui está hard coded mas pode ser "agentificado".
# O agente deve identificar o indexador a ser usado e passar como parâmetro
# Por hora não conseguiremos colocar um agente nessa parte
# Obs: Fica mais fácil se o usuario digitar o indexador...chamar o agente duas
# vezes, uma para tratar dados de MATRICULA outro para tratar os dados de
# SINDICATO, ou se o agente descritor estiver 100% funcional capaz de descrever
# colunas
################################################################################

def consolidar_dados():
  # Com as tabelas arrumadas a consolidacao é procedural
  matriculas_list = []
  if(DataFrameManager.get('DataFrameFinal') is not None):
    DataFrameManager.pop('DataFrameFinal')
  # Loop through each DataFrame in the dictionary
  for key, value in DataFrameManager.items():
      # Check if the DataFrame has a 'MATRICULA' column
      if 'MATRICULA' in value.columns:
          # If it exists, extend the list with the values from that column
          matriculas_list.extend(value['MATRICULA'].tolist())

  # Create a new DataFrame from the list of all matriculas
  df = pd.DataFrame(matriculas_list, columns=['MATRICULA'])

  # Remove duplicate values to get only unique matriculas
  df = df.drop_duplicates()
  print(df.info())


  for src_df in DataFrameManager.values():
      df = simplemerge(df,src_df,'MATRICULA','left')

  df_sindicato = DataFrameManager.get('Base dias uteis')

  df = pd.merge(
      df,
      df_sindicato[['SINDICATO', 'VALOR', 'DIAS UTEIS']],
      on='SINDICATO',
      how='left'
  )
  DataFrameManager.update({'DataFrameFinal': df})



In [ ]:
################################################################################
# Aqui já temos todos os dados na tabela final bastando realizar consolidação,
# padronização, cálculos e limpezas.
# A maioria dessas etapas não seriam bem relizadas pela llm, exceto aquelas em
# que a ferramenta poderia ser melhor aproveitada com a llm para ganhar
# resistência a mudanças em alguns nomes críticos, como MATRICULA virar REGISTRO
# ou coisa similar. (Se bem que la no agente formatador podemos simplesmente
# pedir que seja mantida nossa convenção....
# O que um agente poderia também agregar nessa etapa é recegber do usuário um
# template ou por conversa definir o fomrato do output final.
# Uma outra melhoria seria cada agente responsavel por uma regra de negócio que
# pode ser discutida em tempo de execução por exemplo, antes do resultaod final.
# Mas não cnseguiremos ainda nessa versão.
################################################################################
def iniciar_calculos():
  # Puxar todas as informações para planilha final
  consolidar_dados()
  #Agora consolidar colunas:

  print(consolidar_colunas("DataFrameFinal",'CARGO',['CARGO', 'CARGO_1','CARGO_2', 'CARGO_3'],' | '))
  print(consolidar_colunas("DataFrameFinal",'DESC. SITUACAO',['DESC. SITUACAO','DESC. SITUACAO_1', 'DESC. SITUACAO_2'],' | '))

  df = DataFrameManager.get('DataFrameFinal').copy()

  df.rename(columns={"VALOR_x": "VALOR TOTAL"}, inplace=True)
  df.rename(columns={"VALOR_y": "VALOR DIARIO VR"}, inplace=True)

  # Aqui removemos todos os não elegíveis pelas regras de negócio (exceto profissionais no exterior):
  df = df.loc[
    ~df["CARGO"]
    .astype(str)
    .map(unidecode)
    .str.upper()
    .str.contains("ESTAGIARIO", na=False)
  ]
  df = df.loc[
      ~df["CARGO"]
      .astype(str)
      .map(unidecode)
      .str.upper()
      .str.contains("APRENDIZ", na=False)
  ]
  df = df.loc[
      ~df["DESC. SITUACAO"]
      .astype(str)
      .map(unidecode)
      .str.upper()
      .str.contains("ATESTADO", na=False)
  ]
  df = df.loc[
      ~df["DESC. SITUACAO"]
      .astype(str)
      .map(unidecode)
      .str.upper()
      .str.contains("MATERNIDADE", na=False)
  ]
  df = df.loc[
      ~df["DESC. SITUACAO"]
      .astype(str)
      .map(unidecode)
      .str.upper()
      .str.contains("AUXILIO", na=False)
  ]
  df = df.loc[df['COMUNICADO DE DESLIGAMENTO'] != 'OK']

  # Foi notado que mesmo após todas as remoções e correções alguns não possuiuam info sindical
  df["DIAS UTEIS"] = pd.to_numeric(df["DIAS UTEIS"], errors="coerce").fillna(0)
  df.loc[df["DIAS UTEIS"] == 0, "DIAS UTEIS"] = 22

  df.loc[df["SINDICATO"].isna() | (df["SINDICATO"].astype(str).str.strip() == ""), "OBS GERAL"] = "ATENCAO COLABORADOR SEM INFO SINDICATO. PREENCHENDO COM: DADOS SP"

  df["VALOR DIARIO VR"] = pd.to_numeric(df["VALOR DIARIO VR"], errors="coerce").fillna(0)
  df.loc[df["VALOR DIARIO VR"] == 0, "VALOR DIARIO VR"] = 37.50

  # Agora fazemos os cálculos com as regras definidas
  # interpretando que deverá serfeito calculo proporcional
  # ao dias uteis:
  df["COMPETENCIA"] = "05/2025" # cria coluna competencia
  # 1. Converter datas
  df["ADMISSAO"] = pd.to_datetime(df["ADMISSAO"], errors="coerce", dayfirst=True)
  df["DATA DEMISSAO"] = pd.to_datetime(df["DATA DEMISSAO"], errors="coerce", dayfirst=True)
  df["COMPETENCIA"] = pd.to_datetime("01/" + df["COMPETENCIA"], format="%d/%m/%Y", errors="coerce")

  # 2. Definir início e fim do mês de cálculo
  df["INICIO_MES"] = (df["COMPETENCIA"] - pd.DateOffset(months=1)).apply(lambda x: x.replace(day=1))
  df["FIM_MES"] = df["INICIO_MES"] + MonthEnd(0)

  # 3. Calcular entrada e saída no mês
  df["ENTRADA_MES"] = df[["ADMISSAO", "INICIO_MES"]].max(axis=1)
  df["SAIDA_MES"] = df[["DATA DEMISSAO", "FIM_MES"]].min(axis=1)
  df["SAIDA_MES"] = df["SAIDA_MES"].fillna(df["FIM_MES"])

  # 4. Calcular dias trabalhados no mês
  df["DIAS_TRABALHADOS"] = (df["SAIDA_MES"] - df["ENTRADA_MES"]).dt.days + 1
  df["DIAS_TRABALHADOS"] = df["DIAS_TRABALHADOS"].clip(lower=0)

  # 5. Descontar férias
  df["DIAS DE FERIAS"] = pd.to_numeric(df["DIAS DE FERIAS"], errors="coerce").fillna(0)
  df["DIAS_TRABALHADOS_LIQUIDO"] = df["DIAS_TRABALHADOS"] - df["DIAS DE FERIAS"]
  df["DIAS_TRABALHADOS_LIQUIDO"] = df["DIAS_TRABALHADOS_LIQUIDO"].clip(lower=0)

  # 6. Garantir que DIAS UTEIS seja numérico
  df["DIAS UTEIS"] = pd.to_numeric(df["DIAS UTEIS"], errors="coerce").fillna(0)

  # 7. Calcular total de dias corridos no mês
  df["DIAS_TOTAIS_MES"] = (df["FIM_MES"] - df["INICIO_MES"]).dt.days + 1

  # 8. Calcular dias úteis proporcionais ao tempo trabalhado
  df["DIAS"] = (
      df["DIAS UTEIS"] / df["DIAS_TOTAIS_MES"]
  ) * df["DIAS_TRABALHADOS_LIQUIDO"]

  df["DIAS"] = df["DIAS"].round(0).clip(lower=0)

  # Calculos DE DIAS terminados, vamos jogar o lixo fora
  df.drop(columns=[
    "INICIO_MES", "FIM_MES", "ENTRADA_MES", "SAIDA_MES","EMPRESA",
    "DIAS_TRABALHADOS", "DIAS_TRABALHADOS_LIQUIDO", "DIAS_TOTAIS_MES",
    "DIAS UTEIS", "DIAS DE FERIAS", "DATA DEMISSAO", "COMUNICADO DE DESLIGAMENTO"
  ], inplace=True)

  DataFrameManager.update({'DataFrameFinal': df})
  # As colunas de anotações não vamos jogar fora, mas concatenar tudo na coluna de OBS GERAL:
  consolidar_colunas_como_texto("DataFrameFinal",'OBS GERAL',['OBS GERAL', 'UNNAMED: 2','UNNAMED: 3', 'NA COMPRA?','NA COMPRA?_1','UNNAMED: 3_1','DESC. SITUACAO'],' | ')
  # Agora sim partimos para a etapa final dos cálculos:

  df["DIAS"] = pd.to_numeric(df["DIAS"], errors="coerce").fillna(0)
  df["VALOR DIARIO VR"] = pd.to_numeric(df["VALOR DIARIO VR"], errors="coerce").fillna(0)
  df["VALOR TOTAL"] = pd.to_numeric(df["VALOR TOTAL"], errors="coerce").fillna(0)

  df["VALOR TOTAL"] = df.apply(
      lambda row: row["DIAS"] * row["VALOR DIARIO VR"] if row["VALOR TOTAL"] == 0 else row["VALOR TOTAL"],
      axis=1
  )

  df["Custo empresa"] = df["VALOR TOTAL"] * 0.8
  df["Desconto profissional"] = df["VALOR TOTAL"] * 0.2


  # Por último e não menos importante vaos formatar a saída conforme desejado:
  # Todo: fazer outro agente formatador formatado pra essa etapa:
  df.rename(columns={
    "MATRICULA": "Matricula",
    "ADMISSAO": "Admissão",
    "SINDICATO": "Sindicato do Colaborador",
    "VALOR TOTAL": "TOTAL",
    "COMPETENCIA": "Competência",
    "DIAS": "Dias",
    "VALOR DIARIO VR": "VALOR DIÁRIO VR",
    "OBS GERAL": "OBS GERAL"
  }, inplace=True)

  df = df[df["TOTAL"] != 0]

  df = df[[
      "Matricula", "Admissão", "Sindicato do Colaborador", "Competência", "Dias",
      "VALOR DIÁRIO VR", "TOTAL", "Custo empresa", "Desconto profissional", "OBS GERAL"
  ]]

  # Gera o arquivo excem no DRIVE:
  df.to_excel('VR MENSAL 05.2025.xlsx', index=False)

iniciar_calculos()

<class 'pandas.core.frame.DataFrame'>
Index: 1936 entries, 0 to 2092
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   MATRICULA  1936 non-null   int64
dtypes: int64(1)
memory usage: 30.2 KB
None
[AVISO] Chave 'MATRICULA' não encontrada em ambos os DataFrames.
[AVISO] Chave 'MATRICULA' não encontrada em ambos os DataFrames.
{'status': 'success', 'df_name': 'DataFrameFinal', 'error': None}
{'status': 'success', 'df_name': 'DataFrameFinal', 'error': None}
